In [3]:
import os, sys, email
import numpy as np 
import pandas as pd

from datetime import datetime
import gc
import time

from my_functions import get_user_name

In [4]:
from multiprocessing import Pool
import multiprocessing as mp
from multiprocessing import get_start_method
try: 
    mp.set_start_method('spawn')
except RuntimeError:
    method = get_start_method()
    print('process start method:', method)

nr_of_cpu_cores = mp.cpu_count()
print(nr_of_cpu_cores)

4


In [5]:
# helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append(part.get_payload())

        return ''.join(parts)
    
def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else: 
        addrs = None
    return addrs

In [6]:
def execution_time_str(start_time: time, end_time: time) -> str:
    execution_time = (end_time - start_time) 
    mins = int(execution_time // 60)
    seconds = execution_time % 60
    return f'{mins}mins nd{seconds:.2f}seconds'

In [7]:
from contextlib import contextmanager

@contextmanager
def measurement_of_time(block_name: str=''):
    start_time = time.time()
    yield
    end_time = time.time()
    print(f'block {block_name} executed in: {execution_time_str(start_time, end_time)}')